# Aggregation (집계함수)

In [81]:
import pandas as pd 
import numpy as np 

In [82]:
data = {
    'name': ['김혜수', '박보검', '송혜교', '아이유', '공유', '전지현', '수지', '유재석', '강동원', '윤아'],
    'department': ['인사', '재무', '재무', 'IT', 'IT', '인사', '재무', '인사', 'IT', '재무'],
    'job': ['사원', '과장', '대리', '사원', '팀장', '대리', '대리', '과장', '사원', '팀장'],
    'branch': ['서울', '부산', '서울', '부산', '서울', '서울', '서울', '부산', '서울', '부산'],
    'salary': [3500, 5000, 4200, 3800, 6000, 4500, 4600, 5200, 3900, 6100],
    'hire_date': ['2020-03-15', '2018-07-10', '2019-05-20', '2021-01-10', '2017-09-25', '2019-03-05', '2020-06-12', '2016-11-30', '2022-04-01', '2017-12-15']
}

df = pd.DataFrame(data)

df['hire_date'] = pd.to_datetime(df['hire_date'])    # 날짜형으로 반환하는 함수 

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   name        10 non-null     object        
 1   department  10 non-null     object        
 2   job         10 non-null     object        
 3   branch      10 non-null     object        
 4   salary      10 non-null     int64         
 5   hire_date   10 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 612.0+ bytes


In [83]:
# dataframe 레벨에 대해 사용
df.count()

name          10
department    10
job           10
branch        10
salary        10
hire_date     10
dtype: int64

In [84]:
# series 레벨에 대해 사용
print(df['salary'].size)   # 개수
print(df['salary'].sum())   # 합계
print(df['salary'].mean())  # 평균
print(df['salary'].max())   # 최대값
print(df['salary'].min())   # 최소값
print(df['salary'].median())   # 중위값
print(df['salary'].var())   # 분산
print(df['salary'].std())   # 표준편차

10
46800
4680.0
6100
3500
4550.0
797333.3333333334
892.9352346801718


---

### groupby
- 특정 조건에 맞게 행을 그룹핑하는 매서드
- DataFrameGroupBy 객체 반환

In [85]:
groupby_dept = df.groupby('department')

print(groupby_dept)                  # DataFrameGroupBy object
print(groupby_dept.groups)           # 딕셔너리 형태로 반환 : 키값=컬럼명 / 벨류값=해당 로우의 인덱스넘버     # groupby 현황 반환
print(type(groupby_dept))            # DataFrameGroupBy object

{'IT': [3, 4, 8], '인사': [0, 5, 7], '재무': [1, 2, 6, 9]}
<class 'pandas.core.groupby.generic.DataFrameGroupBy'>


In [86]:
# 숫자형 데이터 -> 그룹별 집계함수 조회
print(groupby_dept['salary'].mean())
print(groupby_dept['salary'].max())
print(groupby_dept['salary'].min())

department
IT    4566.666667
인사    4400.000000
재무    4975.000000
Name: salary, dtype: float64
department
IT    6000
인사    5200
재무    6100
Name: salary, dtype: int64
department
IT    3800
인사    3500
재무    4200
Name: salary, dtype: int64


In [87]:
# 숫자형 데이터가 아닌 경우 -> count()   (int가 아닌경우, 통계량 산출 제한)
print(groupby_dept['job'].count())           # 총 개수 반환
print(groupby_dept['job'].value_counts())    # 그룹 안에서 value값에 대한 count 반환

department
IT    3
인사    3
재무    4
Name: job, dtype: int64
department  job
IT          사원     2
            팀장     1
인사          과장     1
            대리     1
            사원     1
재무          대리     2
            과장     1
            팀장     1
Name: count, dtype: int64


In [88]:
# 날짜형 데이터
print(groupby_dept['hire_date'].min())     # 가장 과거인 날짜(입사일이 가장 빠른)
print(groupby_dept['hire_date'].max())     # 가장 최근인 날짜(입사일이 가장 늦은)

department
IT   2017-09-25
인사   2016-11-30
재무   2017-12-15
Name: hire_date, dtype: datetime64[ns]
department
IT   2022-04-01
인사   2020-03-15
재무   2020-06-12
Name: hire_date, dtype: datetime64[ns]


In [89]:
# named aggregation
# - DataFrameGroupBy의 여러 열에 대해 다양한 집계 함수를 적용하면서 결과 열 이름을 지정

groupby_dept.agg(
    salary_mean = ('salary', 'mean'),    # 열, 통계량 지정
    salary_max = ('salary', 'max'), 
    hire_date_min = ('hire_date', 'min'),
    hire_date_max = ('hire_date', 'max')
)

,salary_mean,salary_max,hire_date_min,hire_date_max
department,,,,
IT,4566.666667,6000,2017-09-25,2022-04-01
인사,4400.000000,5200,2016-11-30,2020-03-15
재무,4975.000000,6100,2017-12-15,2020-06-12


In [90]:
# 두 개 이상의 컬럼으로 그룹핑
groupby_dept_job = df.groupby(['department', 'job'])
groupby_dept_job.groups

{('IT', '사원'): [3, 8], ('IT', '팀장'): [4], ('인사', '과장'): [7], ('인사', '대리'): [5], ('인사', '사원'): [0], ('재무', '과장'): [1], ('재무', '대리'): [2, 6], ('재무', '팀장'): [9]}

In [91]:
groupby_dept_job.count()   # 각 로우의 컬럼개수로 카운트 

name  branch  salary  hire_date
department job                                 
IT         사원      2       2       2          2
           팀장      1       1       1          1
인사         과장      1       1       1          1
           대리      1       1       1          1
           사원      1       1       1          1
재무         과장      1       1       1          1
           대리      2       2       2          2
           팀장      1       1       1          1

In [92]:
groupby_dept_job.value_counts()    # 각각의 값을 반환

department  job  name  branch  salary  hire_date 
IT          사원   강동원   서울      3900    2022-04-01    1
                 아이유   부산      3800    2021-01-10    1
            팀장   공유    서울      6000    2017-09-25    1
인사          과장   유재석   부산      5200    2016-11-30    1
            대리   전지현   서울      4500    2019-03-05    1
            사원   김혜수   서울      3500    2020-03-15    1
재무          과장   박보검   부산      5000    2018-07-10    1
            대리   송혜교   서울      4200    2019-05-20    1
                 수지    서울      4600    2020-06-12    1
            팀장   윤아    부산      6100    2017-12-15    1
Name: count, dtype: int64

In [93]:
for (name, group) in groupby_dept_job :      # 순회해서 값 반환하는 방법 
    display(name)                            # print와 동일. 이쁘게 보이는거임. (주피터 전용)
    display(group)

('IT', '사원')

,name,department,job,branch,salary,hire_date
3,아이유,IT,사원,부산,3800,2021-01-10
8,강동원,IT,사원,서울,3900,2022-04-01


('IT', '팀장')

,name,department,job,branch,salary,hire_date
4,공유,IT,팀장,서울,6000,2017-09-25


('인사', '과장')

...

,name,department,job,branch,salary,hire_date
1,박보검,재무,과장,부산,5000,2018-07-10


('재무', '대리')

,name,department,job,branch,salary,hire_date
2,송혜교,재무,대리,서울,4200,2019-05-20
6,수지,재무,대리,서울,4600,2020-06-12


('재무', '팀장')

,name,department,job,branch,salary,hire_date
9,윤아,재무,팀장,부산,6100,2017-12-15


---

### pivot_table
- 특정 컬럼(인덱스, 컬럼)을 기준으로 값 컬럼을 열람하는 기능
- 연관 있는 데이터를 가독성 좋게 출력 가능

In [94]:
data = {
    'Date': ['2024-10-11', '2024-10-12', '2024-10-13', '2024-10-14', '2024-10-14'],
    'City': ['서울', '인천', '평택', '화성', '서울'],
    'Temperature': [17, 18, 19, 18, 19]
}

df =pd.DataFrame(data)
df

,Date,City,Temperature
0,2024-10-11,서울,17
1,2024-10-12,인천,18
2,2024-10-13,평택,19
3,2024-10-14,화성,18
4,2024-10-14,서울,19


In [95]:
df.pivot_table(index='Date', columns='City', values='Temperature')                     # (index='', columns='', values='') 기본 지정
                                                                                       # 피벗테이블 형식으로 변환하는 함수 

City,서울,인천,평택,화성
Date,,,,
2024-10-11,17.0,NaN,NaN,NaN
2024-10-12,NaN,18.0,NaN,NaN
2024-10-13,NaN,NaN,19.0,NaN
2024-10-14,19.0,NaN,NaN,18.0


In [96]:
# 제품별 판매데이터 
data = {
    '날짜': [
        '2024-01-01', '2024-01-01', '2024-01-01', '2024-01-01',
        '2024-01-02', '2024-01-02', '2024-01-02', '2024-01-02',
        '2024-01-03', '2024-01-03'
    ],
    '상품': [
        '아이폰 14', '갤럭시 S23', '아이패드 프로', '아이폰 14',
        '아이폰 14', '갤럭시 S23', '아이패드 프로', '갤럭시 S23',
        '맥북 프로', '아이폰 14'
    ],
    '판매량': [15, 30, 5, 10, 20, 25, 10, 5, 8, 12],
    '가격': [1000, 900, 800, 1000, 1000, 900, 800, 900, 2000, 1000]
}

df2 =pd.DataFrame(data)
df2

,날짜,상품,판매량,가격
0,2024-01-01,아이폰 14,15,1000
1,2024-01-01,갤럭시 S23,30,900
2,2024-01-01,아이패드 프로,5,800
3,2024-01-01,아이폰 14,10,1000
4,2024-01-02,아이폰 14,20,1000
5,2024-01-02,갤럭시 S23,25,900
6,2024-01-02,아이패드 프로,10,800
7,2024-01-02,갤럭시 S23,5,900
8,2024-01-03,맥북 프로,8,2000
9,2024-01-03,아이폰 14,12,1000


In [97]:
df2.pivot_table(
    index='날짜', columns='상품', values='판매량',          # 구조를 바꾸는 과정에서 수치값들의 평균을 반환함. 
    fill_value=0,   #결측치 대체값
    aggfunc='sum',                                        # 기본값 = 평균에서 합계로 변경. 구조 변경 시 중복값이 발생할 때,  처리 방법을 지정
    margins=True                                          # 행, 열 방향의 합계를 추가 
)

상품,갤럭시 S23,맥북 프로,아이패드 프로,아이폰 14,All
날짜,,,,,
2024-01-01,30,0,5,25,60
2024-01-02,30,0,10,20,60
2024-01-03,0,8,0,12,20
All,60,8,15,57,140


---

### 실습

### **[실습 1] 데이터 정리 및 전처리**

다음은 직장 내 직원 데이터입니다. 결측된 값들이 포함되어 있습니다. `Salary`와 `YearsAtCompany`의 결측값을 각 열의 평균값으로 채우세요.

In [98]:
employee_data = {
    'EmployeeID': [101, 102, 103, 104, 105],
    'Name': ['John', 'Emily', 'Sara', 'Tom', 'Anna'],
    'Department': ['HR', 'IT', 'Marketing', None, 'HR'],
    'Salary': [50000, 70000, None, 45000, 52000],
    'YearsAtCompany': [5, None, 8, 3, 4]
}

df = pd.DataFrame(employee_data)

# print(df.info())
# print(df[df.duplicated(['Department'])])
print(df.isna().sum())

print(df['Salary'].mean())
print(df['YearsAtCompany'].mean())

df['Salary'] = df['Salary'].fillna('54250')
df['YearsAtCompany'] = df['YearsAtCompany'].fillna('5')

print(df.info())


EmployeeID        0
Name              0
Department        1
Salary            1
YearsAtCompany    1
dtype: int64
54250.0
5.0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   EmployeeID      5 non-null      int64 
 1   Name            5 non-null      object
 2   Department      4 non-null      object
 3   Salary          5 non-null      object
 4   YearsAtCompany  5 non-null      object
dtypes: int64(1), object(4)
memory usage: 332.0+ bytes
None


### [실습 2] **중복값 제거**

다음은 한 판매점의 제품 판매 기록입니다. 동일한 제품이 중복으로 기록되어 있습니다. 중복된 `ProductID`를 가진 제품의 중복된 행을 제거한 후, 가장 저렴한 가격을 가진 데이터만 남기세요.

In [99]:
product_data = {
    'ProductID': [1, 2, 3, 1, 4, 2],
    'ProductName': ['Laptop', 'Mouse', 'Monitor', 'Laptop', 'Keyboard', 'Mouse'],
    'Price': [1200, 25, 300, 1150, 45, 22],
    'QuantitySold': [10, 100, 50, 8, 70, 120]
}

df = pd.DataFrame(product_data)

In [100]:
df.info()

# df.duplicated().sum() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ProductID     6 non-null      int64 
 1   ProductName   6 non-null      object
 2   Price         6 non-null      int64 
 3   QuantitySold  6 non-null      int64 
dtypes: int64(3), object(1)
memory usage: 324.0+ bytes


### [실습 3] **데이터 정렬**

다음은 자동차 모델별 성능 데이터를 포함하고 있습니다. `마력(horsepower)`을 기준으로 오름차순으로 데이터를 정렬한 후, `연비(mpg)`가 높은 상위 3개 자동차만 출력하세요.

In [101]:
car_data = {
    'Model': ['Sedan', 'SUV', 'Truck', 'Coupe', 'Convertible'],
    'Horsepower': [180, 250, 200, 170, 150],
    'MPG': [30, 20, 15, 35, 28],
    'Price': [30000, 45000, 40000, 28000, 32000]
}

### [실습 4] **데이터 필터링**

다음은 축구 선수들의 데이터를 나타냅니다. 필터링을 통해 포지션이 "Forward"이고, 나이가 30 이하인 선수만 출력하세요.

In [102]:
football_data = {
    'PlayerID': [201, 202, 203, 204, 205],
    'Name': ['Ronaldo', 'Messi', 'Neymar', 'Mbappe', 'Kane'],
    'Position': ['Forward', 'Forward', 'Midfielder', 'Forward', 'Forward'],
    'GoalsScored': [600, 700, 400, 150, 200],
    'Age': [35, 33, 29, 22, 27]
}